In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict
import simpler_sf
simpler_sf.simple_salesforce()
import simple_salesforce

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
#nombreArchivoHost = "VentaCARE_ASSISTANCE_" + '31102023' + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2023-11-13
13112023
VentaCARE_ASSISTANCE_13112023.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_13112023.xlsx


In [5]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [6]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [7]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [8]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape


c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(6273, 17)

In [9]:
archivoBCI.dtypes


POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [10]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,16368915,4,FEMENINO,TITULAR MASCULINO,TANIA REYES SANCHEZ,1987-05-05,tania.reyes.s1987@gmail.com,SN,950999267
1,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,27877512,7,FEMENINO,HIJO,FRANCISCO ALONSO PERES REYES,2022-08-02,,SN,0
2,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,25837523,8,FEMENINO,HIJA,JULIETA FERNANDA PERES REYES,2017-07-18,,SN,0
3,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,23407616,7,FEMENINO,HIJA,RENATA MAGDALENA PERES REYES,2010-08-28,,SN,0
4,2602260,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,11164052,1,11164052,1,MASCULINO,TITULAR MASCULINO,NESTOR VIDAL BASCUNAN,1967-03-07,vidal.nestor67@gmail.com,PEDRO MONTT 980,982339878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6218,2623145,2023-07-26,2024-07-25,SAIN,M6,SALUD INDIVIDUAL PLAN 80 OV,10526870,K,10526870,K,MASCULINO,TITULAR MASCULINO,RICARDO MARIO MANSILLA SERRANO,1969-05-22,rmansilla@vidatres.cl,SANTIAGO,954214843
6219,2623148,2023-07-26,2024-07-25,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,13247174,6,13247174,6,FEMENINO,TITULAR MASCULINO,MARIA JOSEFINA ELIZALDE VEGA,1977-03-01,MAJOEV@HOTMAIL.COM,CURICO 372,981371593
6220,2623149,2023-07-26,2024-07-25,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,16525568,2,16525568,2,MASCULINO,TITULAR MASCULINO,ALFREDO ALEXIS ARANEDA ROA,1987-08-12,ALFREDO.ARANEDA.ROA@GMAIL.COM,NO INFORMADO,921687636
6221,2647198,2023-09-08,2024-09-07,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,12671305,3,12671305,3,FEMENINO,TITULAR FEMENINO,SALINAS HERRERA MARCELA ALEJANDRA,1973-11-08,msalinas@mcpromociones.cl,albacete 4420 departamento 86,998240188


In [11]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [12]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11252\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [13]:
archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,16368915,4,...,950999267,163689154,163689154,2602258,Mujer,tania.reyes.s1987@gmail.com,,950999267,Tania Reyes,Reyes Sanchez
1,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,27877512,7,...,,163689154,278775127,2602258,Mujer,,,,Francisco Alonso,Peres Reyes
2,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,25837523,8,...,,163689154,258375238,2602258,Mujer,,,,Julieta Fernanda,Peres Reyes
3,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,23407616,7,...,,163689154,234076167,2602258,Mujer,,,,Renata Magdalena,Peres Reyes
4,2602260,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,11164052,1,11164052,1,...,982339878,111640521,111640521,2602260,Hombre,vidal.nestor67@gmail.com,Pedro Montt 980,982339878,Nestor Vidal,Vidal Bascunan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6218,2623145,2023-07-26,2024-07-25,SAIN,M6,SALUD INDIVIDUAL PLAN 80 OV,10526870,K,10526870,K,...,954214843,10526870K,10526870K,2623145,Hombre,rmansilla@vidatres.cl,Santiago,954214843,Ricardo Mario,Mansilla Serrano
6219,2623148,2023-07-26,2024-07-25,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,13247174,6,13247174,6,...,981371593,132471746,132471746,2623148,Mujer,majoev@hotmail.com,Curico 372,981371593,Maria Josefina,Elizalde Vega
6220,2623149,2023-07-26,2024-07-25,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,16525568,2,16525568,2,...,921687636,165255682,165255682,2623149,Hombre,alfredo.araneda.roa@gmail.com,,921687636,Alfredo Alexis,Araneda Roa
6221,2647198,2023-09-08,2024-09-07,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,12671305,3,12671305,3,...,998240188,126713053,126713053,2647198,Mujer,msalinas@mcpromociones.cl,Albacete 4420 Departamento 86,998240188,Salinas Herrera,Marcela Alejandra


In [14]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [15]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """


' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [16]:
results = sf.query_all(
    """
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Canal Individual'
   """
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["sfRut",]]


pacienteProgr.shape


attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtphAAC')])
IdentificationId__pc -> 196364803
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpiAAC')])
IdentificationId__pc -> 154512675
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpjAAC')])
IdentificationId__pc -> 243180856
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpkAAC')])
IdentificationId__pc -> 14067401K
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtplAAC')])
IdentificationId__pc -> 268701028
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtsvAAC')])
IdentificationId__pc -> 101272702
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/A

(2685, 1)

In [17]:
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].str.replace(" ", "")
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(pacienteProgr["sfRut"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]

archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI.drop_duplicates(["Rut Carga", "Rut Titular"], keep="last")

archivoBCI = archivoBCI[
    ["Rut Carga", "sexo", "direccion", "email", "celular", "nombres", "apellidos"]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
""" archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
) """


archivoBCI.columns


Index(['IdentificationId__pc', 'HealthCloudGA__Gender__pc', 'BillingStreet',
       'PersonEmail', 'Phone', 'FirstName', 'LastName', 'BillingCountry',
       'Identification_Type__c', 'Pa_s_del_Paciente__c', 'RecordTypeId',
       'Cuenta_validada__c', 'Validaci_n__c', 'Email_Seteado__c'],
      dtype='object')

In [18]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """
soql_query = """ select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Canal Individual' """
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
df = df.drop(columns="Id")
list_of_records = df.to_dict("index")

""" derty = sf.quick_search("153298939")
print(derty) """
list_of_records[0]["Paciente__r"]["IdentificationId__pc"]

'196364803'

Weeeeeeeeeeeenooo


In [19]:
# rut = "124942012"
def traerId(rut):
    if (len(rut)>=8):
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""        

In [20]:
archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(lambda y: traerId(y))
archivoBCI["Programas_por_Empresas__c"] = "a1W8c000008TCWxEAO"
archivoBCI.head()

vacio
vacio
vacio
0018c00002eg3icAAA 136911074
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002bKV6eAAG 124633435
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002eVKFFAA4 82439382
0018c00002eTmapAAC 10356235K
vacio
0018c00002eh0iSAAQ 167878156
vacio
0018c00002ej241AAA 263524772
0018c00002ej240AAA 172529569
0018c00002ej23zAAA 24042651K
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002eUJAIAA4 16161288K
0018c00002eUz4TAAS 239419801
0018c00002eV5a1AAC 252654259
0018c00002eVD6kAAG 27154992K
0018c00002eUCmwAAG 153164452
vacio
vacio
vacio
0018c00002cVSGAAA4 100734036
vacio
vacio
0018c00002axcUGAAY 201774209
vacio
0018c00002cVoBFAA0 143426696
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002eTmUuAAK 103365090
vacio
0018c00002mRs3RAAS 90505750
vacio
vacio
vacio


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
3958,151725856,Hombre,"Los Canelos 283, Coelemu",felipesanmartinorue@gmail.com,994371254,Felipe Andre,Martin Orue,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
3959,239006957,Mujer,,,,Seplveda Antonella,San Martn,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
3960,139039475,Mujer,Santiago,,,Karla Francisca,Riquelme Vargas,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
3961,136911074,Hombre,,,,Julio Cesar,Valdes Rodriguez,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,0018c00002eg3icAAA,a1W8c000008TCWxEAO
3962,264940346,Mujer,,,,Gracia Isabel,Valds Riquelme,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO


In [21]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False)

archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
3958,151725856,Hombre,"Los Canelos 283, Coelemu",felipesanmartinorue@gmail.com,994371254,Felipe Andre,Martin Orue,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
3959,239006957,Mujer,,,,Seplveda Antonella,San Martn,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
3960,139039475,Mujer,Santiago,,,Karla Francisca,Riquelme Vargas,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
3961,136911074,Hombre,,,,Julio Cesar,Valdes Rodriguez,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,0018c00002eg3icAAA,a1W8c000008TCWxEAO
3962,264940346,Mujer,,,,Gracia Isabel,Valds Riquelme,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO


In [22]:
prueba = "153298939"
sf.query(
    format_soql(
        "SELECT Id FROM Account WHERE IdentificationId__pc = {last_name}",
        last_name=prueba,
    )
)

OrderedDict([('totalSize', 1),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])),
                            ('Id', '0018c00002anTmeAAE')])])])

In [23]:
""" sf.Account.create(
    {
        "IdentificationId__pc": "12345678910",
        "HealthCloudGA__Gender__pc": "Mujer",
        "FirstName": "johan",
        "LastName": "Smith",
        "PersonEmail": "example@example.com",
        "BillingCountry": "Chile",
        "Identification_Type__c": "RUT",
        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",
        "RecordTypeId": "0128c000002Byp4AAC",
    }
)
 """

# sf.Account.update(1, {'Name': 'Test Account', ContactId: contact_id}) # doctest: +SKIP

' sf.Account.create(\n    {\n        "IdentificationId__pc": "12345678910",\n        "HealthCloudGA__Gender__pc": "Mujer",\n        "FirstName": "johan",\n        "LastName": "Smith",\n        "PersonEmail": "example@example.com",\n        "BillingCountry": "Chile",\n        "Identification_Type__c": "RUT",\n        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",\n        "RecordTypeId": "0128c000002Byp4AAC",\n    }\n)\n '

In [24]:
""" import pandas as pd
sf.bulk.Account.query("SELECT Id, Email FROM Account")

df = pd.DataFrame.from_dict(data,orient='columns').drop('attributes',axis=1) """

import pandas as pd

df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id, Email FROM Account WHERE Id IN ({})", dataframe_to_sfdc_list(df,column)))


NameError: name 'column' is not defined

In [ ]:
contact = sf.Account.get('0018c00002amLcCAAU')

In [ ]:
# sf = simple_salesforce.Salesforce(...)

query = "SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Account.IdentificationId__pc, Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c,Empresa_Principal__c,Country from ServiceAppointment"

df = sf.smart_query(query)
print(df)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:20<00:00, 20.23s/it]

                       Id           AccountId    CreatedDate     RUT__c  \
0      08p8c0000007756AAA  0018c00002ay9PHAAY  1668868174000  150066417   
1      08p8c000000775GAAQ  0018c00002amLcAAAU  1668870893000   78143835   
2      08p8c000000775LAAQ  0018c00002amLcVAAU  1668871031000  18674344K   
3      08p8c0000007769AAA  0018c00002amLcZAAU  1668873317000  106879508   
4      08p8c00000077CcAAI  0018c00002bJgIaAAK  1669045284000  248270454   
...                   ...                 ...            ...        ...   
38396  08pRN0000005D1RYAU  0018c00002cVJb4AAG  1698885404000  20415015K   
38397  08pRN0000005D9WYAU  0018c00002egprZAAQ  1698888851000  128434097   
38398  08pRN0000005DPdYAM  0018c00002egmXUAAY  1698925596000  201508940   
38399  08pRN0000005DSrYAM  001RN000000qoi8YAA  1698927382000  153712905   
38400  08pRN0000005DUTYA2  001RN000001vnVeYAI  1698927549000  203630832   

      Account.IdentificationId__pc   Work_TypeGroup__c      Status  \
0                        1500

In [ ]:
df.head(-50)

,Id,AccountId,CreatedDate,RUT__c,Account.IdentificationId__pc,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c,Empresa_Principal__c,Country
0,08p8c0000007756AAA,0018c00002ay9PHAAY,1668868174000,150066417,150066417,0VS8c0000005YaAGAU,Cancelado,Ingreso,SA-0030,0058c00000DcdNgAAJ,Phone,-,Chile
1,08p8c000000775GAAQ,0018c00002amLcAAAU,1668870893000,78143835,78143835,0VS8c0000005YaAGAU,Ninguno,Seguimiento,SA-0032,0058c00000DcdNgAAJ,Phone,-,Chile
2,08p8c000000775LAAQ,0018c00002amLcVAAU,1668871031000,18674344K,18674344K,0VS8c0000005YZvGAM,Ninguno,Ingreso,SA-0033,0058c00000DcdO3AAJ,Phone,-,Chile
3,08p8c0000007769AAA,0018c00002amLcZAAU,1668873317000,106879508,106879508,0VS8c0000005YaeGAE,Ninguno,Seguimiento,SA-0045,0058c00000DcdNkAAJ,Phone,0018c00002axQecAAE,Chile
4,08p8c00000077CcAAI,0018c00002bJgIaAAK,1669045284000,248270454,248270454,0VS8c0000005YaAGAU,Cancelado,Ingreso,SA-0059,0058c00000DcdNhAAJ,Phone,-,Chile
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38346,08pRN00000054xRYAQ,0018c00002ehvbxAAA,1698786633000,165198638,165198638,0VS8c000000obNUGAY,Programado,Seguimiento MetLife,SA-43357,0058c00000DcdOCAAZ,Phone,0018c00002kBEx5AAG,Chile
38347,08pRN00000054z3YAA,0018c00002ay11SAAQ,1698786655000,240398990,240398990,0VS8c000000obNUGAY,Programado,Seguimiento MetLife,SA-43358,0058c00000DcdOnAAJ,Phone,0018c00002axQcqAAE,Chile
38348,08pRN000000550fYAA,0018c00002ehvbxAAA,1698786696000,165198638,165198638,0VS8c000000obNUGAY,Programado,Seguimiento MetLife,SA-43359,0058c00000DcdOCAAZ,Phone,0018c00002kBEx5AAG,Chile
38349,08pRN000000552HYAQ,0018c00002ehvbxAAA,1698786735000,165198638,165198638,0VS8c000000obNUGAY,Programado,Seguimiento MetLife,SA-43360,0058c00000DcdOCAAZ,Phone,0018c00002kBEx5AAG,Chile


In [ ]:
rut = "153298939"

""" busquedaDatos = (
    "FIND {" + rut + "} "
) """
derty = sf.quick_search("124942012")
derty


OrderedDict([('searchRecords',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Account/0018c00002eh9zgAAA')])),
                            ('Id', '0018c00002eh9zgAAA')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Contact'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Contact/0038c000039H8yDAAS')])),
                            ('Id', '0038c000039H8yDAAS')])])])

In [ ]:
df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
archivoBCI["IdentificationId__pc"]
df.head()

,Id
0,1
1,2
2,3


In [ ]:
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id FROM Account WHERE IdentificationId__pc IN ({})", dataframe_to_sfdc_list(pacienteProgr,"sfRut")))

SalesforceGeneralError: Error Code 414. Response content: <h1>Bad Message 414</h1><pre>reason: URI Too Long</pre>

In [ ]:
pacienteProgr.head()

,sfRut
0,196364803
1,154512675
2,243180856
3,14067401K
4,268701028


Codigo que dio github Copilot para la solucion del error de 0 en envio de correos

In [ ]:
import win32com.client as win32
import os
import shutil

nombre_reporte = "Vida Security - Ferronor"
nombre_carpeta = "F:/Reportes SF/Vida Security/"
nombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

# create Excel application object
excel = win32.gencache.EnsureDispatch('Excel.Application')

# set ScreenUpdating property to False to hide Excel application
excel.ScreenUpdating = False

# open workbook
workbook = excel.Workbooks.Open(nombre_archivo_final)

# do some work with the workbook

# save and close workbook
workbook.Save()
workbook.Close()

# set ScreenUpdating property to True to show Excel application
excel.ScreenUpdating = True

# quit Excel application
excel.Quit()

# release COM object
del excel

envio_de_email(
    destinatarios=["marialaura.perez@security.cl",],
    concopia=[
        "dnunez@careassistance.com",
        "dgarcia@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx",
    activo=1,
)

In [ ]:
import win32com.client as win32
import os
import shutil
import datetime as dt

def crear_carpeta(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)

def actualizar(Archivo_origen):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(Archivo_origen)
    wb.RefreshAll()
    wb.Save()
    wb.Close()

def envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ";".join(destinatarios)
    mail.CC = ";".join(concopia)
    mail.Subject = asunto
    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"
    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)
    mail.Send()

nombre_reporte = "Zurich - AMSA"
nombre_carpeta = "F:/Reportes SF/Zurich/"
nombre_archivo_final = nombre_carpeta + str(dt.date.today()) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dt.date.today())
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

envio_de_email(
    destinatarios=[
        "clara.angelica.munoz@zurich.com",
        "claudia.ivonne.zapata@zurich.com",
        "maria.moreno.poblete@zurich.com",
        "nadia.pino@zurich.com",
    ],
    concopia=[
        "dgarcia@careassistance.com",
        "dnunez@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx"